# TP2

In [ ]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np

In [ ]:
#sc.stop()
sc = pyspark.SparkContext('local[*]')

In [ ]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)

In [ ]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')

In [ ]:
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)

In [ ]:
test = pd.read_csv("Sources/test_final_100k.csv")

In [ ]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()


In [ ]:
postulaciones.fillna(-1)[postulaciones.fillna(-1)['FEM']==-1].shape

In [ ]:
resultado = postulaciones['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [ ]:
test = pd.concat([test,postulaciones])

In [ ]:
test.head()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not postulaciones[col].dtype == 'O': continue
    #postulaciones[col] = postulaciones.apply(lambda x: str(x[col]),axis=1)
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])
    #postulaciones[col] = le.transform(postulaciones[col])

In [ ]:
#postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [ ]:

postulaciones = test[100000:]
test = test[:100000]

In [ ]:
postulaciones.head()

In [ ]:
test['sepostulo'] = 1
resu_test = test['sepostulo']
test.drop('sepostulo',axis=1,inplace=True)

In [ ]:
aviso = None
postulantes = None
import xgboost as xgb
# read in data
print('Creando Matriz')
dtrain = xgb.DMatrix(data=postulaciones,label=resultado)
dtest = xgb.DMatrix(data=test)
# specify parameters via map
print('Entrenando')
param = {'max_depth':50, 'eta':0.5, 'silent':1, 'objective':'multi:softmax','num_class':2, 'subsample':1}
num_round = 50

bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

In [ ]:
preds[preds==1]

In [ ]:
test = pd.read_csv("Sources/test_final_100k.csv")



In [ ]:
test['sepostulo'] = preds

In [ ]:
test.head()

In [ ]:
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)

In [ ]:
test.to_csv('Sources/resultados.csv', index=False)

In [ ]:
pd.concat([test['idaviso'],postulaciones['idaviso']])